### 1.走流程图

In [ ]:
def find_best_candidates(row,data_base,new_in):
    test_index = row.name
    
    # 创建训练集副本
    candidates = data_base.copy(deep = False)
    candidates["type"] = 0
    candidates["info"] = 0
    candidates["plant"] = 0
    candidates["number"] = 0
    candidates["state"] = 0
    candidates["ord"] = 0
    candidates["fg&rp"] = 0
    candidates["newin"] = 0
    candidates["pos"] = 0


    # 1. type比对
    type = row["TYPE"]
    info = row["INFO"]
    # type相等
    myindex_1 = candidates.index[
        candidates["TYPE"].isin(
            list(df_cos_sim.columns[abs(df_cos_sim.loc[type, :] - 1) <= 0.000001])
        )
    ]
    myindex_1_1 = myindex_1[candidates.loc[myindex_1, "INFO"] == info]
    # myindex_1_2 = myindex_1[candidates.loc[myindex_1, "INFO"] != info]
    candidates.loc[myindex_1, "type"] += 1
    candidates.loc[myindex_1_1, "info"] += 1

    # type不相等
    myindex_2 = candidates.index[
        candidates["TYPE"].isin(
            list(df_cos_sim.columns[abs(df_cos_sim.loc[type, :] - 1) > 0.000001])
        )
    ]
    candidates.loc[myindex_2, "type"] = df_cos_sim.loc[
        type, candidates.loc[myindex_2, "TYPE"]
    ].values

    # fg&pos比对
    fgrp = row["FGRP"]
    pos = row["POS"]
    '''
    candidates.loc[
        (candidates["FGRP"] == fgrp) & (candidates["POS"] == pos), "fg&rp"
    ] += 1
    '''
    candidates.loc[
   (candidates["POS"] == pos), "pos"
    ] += 1 
    
    candidates.loc[
   (candidates["FGRP"] == fgrp), "fg&rp"
    ] += 1 

    # number
    num = row['NUMBER']
    candidates.loc[
        (candidates["NUMBER"] == num), "number"
    ] += 1


    # newin比对
    newin = new_in
    myindex_4 = candidates.index

    myindex_4_1 = candidates.loc[myindex_4].index[
        candidates.loc[myindex_4, "NEWIN"].isin(newin)
    ]
    candidates.loc[myindex_4_1, "newin"] += 1

    # PLANT
    plant = row["PLANT"]
    candidates.loc[candidates["PLANT"] == plant, "plant"] += 1

    plant_list = [
        "CHN03",
        "DAQ03",
        "LUQ03",
        "CHA03",
        "BP2TC",
        "BP2TH",
        "BS8CA",
        "BSNRA",
    ]

    candidates.loc[
        (candidates["PLANT"].isin(plant_list)) & (candidates["PLANT"] != plant), "plant"
    ] += 0.8

    # state比对
    states = ["P", "I"]
    candidates.loc[candidates["S"].isin(states), "state"] += 1

    # ord比对
    candidates.loc[candidates["ORD"] == "MO", "ord"] += 1

    #PSS比对


    # 计算最终得分
    candidates["final_score"] = (
        0.55 * candidates["type"]
        + 0.15 * candidates["info"]
        + 0.08 * candidates["fg&rp"]
        +0.04*candidates["pos"]
        + 0.18 * candidates["number"]
    ) * 0.8 + (
        0.47 * candidates["newin"]
        + 0.47 * candidates["plant"]
        + 0.03 * candidates["ord"]
        + 0.03 * candidates["state"]
    ) * 0.2
    final = candidates.sort_values(by="final_score", ascending=False)
    final_output = final.iloc[:3, :]
    
    print(row.name)

    return final_output.reset_index(drop=True)

### 2.data_base预处理
1.删除剔除UL=0，UL或者LC或者长宽高为空/1的数据

2.剔除状态错误数据（EXP=Y/S=N）

3.将NAME分解为TYPE-INFO，若有多个-，按照第一个-进行分解

In [ ]:
def data_base_preprocess(data_base,test_newin=""):
    
    '''
    1. data_base:读取的VSIM表格
    2.test_newin默认空，检测数据的项目号，需要在data_base中删除
    '''
    
    #提取所需要的列
    data_base = data_base[["NUMBER","NEWIN","NAME","PSS","PLANT","DRAWING NO","EXP","ORD","REQ","S","STATE","LC","UL","LENGTH(MM)","WIDTH(MM)","HEIGHT(MM)","EMB1","FGRP","POS"]]
    #删掉test检测数据
    data_base = data_base[data_base["NEWIN"] !=test_newin]
    
    # 通过NAME拆分INFO TYPE
    data_base["NAMELIST"] = data_base["NAME"].str.rsplit("-", n=0)
    
    data_base["TYPE"] = data_base["NAMELIST"].str[0].str.rsplit(",").str[0]
    data_base["INFO"] = data_base["NAMELIST"].str[1]
    
    #剔除UL=0，UL或者LC或者长宽高为空的数据
    data_base = data_base.loc[-( (data_base["UL"] == 0) | (data_base["UL"].isna())| (data_base["LENGTH(MM)"].isna() | data_base["LC"].isna()) | (data_base["LENGTH(MM)"]*data_base["WIDTH(MM)"]*data_base["HEIGHT(MM)"]==1) | (data_base["LENGTH(MM)"]*data_base["WIDTH(MM)"]*data_base["HEIGHT(MM)"]==0))]
    data_base = data_base.loc[-((data_base["EXP"] == "Y") | (data_base["S"] == "N"))]
    
    return data_base

### 3.test预处理
将NAME分解为TYPE-INFO，若有多个-，按照第一个-进行分解

In [ ]:
def test_preprocess(axa,single_bom,plant):
        
        test1 = axa[["PART NO","PART NAME","PART DESCRIPTION"]]
        test2 = single_bom[["Fgrp","Pos ","PSS","-Partno-","Description-------"]]
        test1[["PART NAME","PART DESCRIPTION"]] = test1[["PART NAME","PART DESCRIPTION"]].apply(lambda x : x.apply(lambda y:str(y).strip()))
        test2["Description-------"] = test2["Description-------"].apply(lambda y:str(y).strip())
        test = pd.merge(test1,test2,how = "right",left_on = "PART NO",right_on = "-Partno-")
        
        test["NAME"] = test["PART NAME"]+"-"+test["PART DESCRIPTION"]
        test["NAME"] = test["NAME"].apply(lambda x : str(x).replace(">",""))
        
        test["NAMELIST"] = test["NAME"].str.rsplit("-", n=0)
        
        test["TYPE"] = test["NAMELIST"].str[0].str.rsplit(",").str[0]
        test["INFO"] = test["NAMELIST"].str[1]
        
        test.rename(columns = {"Fgrp":"FGRP","Pos ":"POS","PART NO":"NUMBER"},inplace = True)
        
        test.dropna(subset = ["NAME","FGRP","POS","NUMBER"],inplace = True)
        
        test["PLANT"] = plant
        
        return test[["NUMBER","NAME","PLANT","TYPE","INFO","FGRP","POS","PSS"]].drop_duplicates().reset_index(drop=True)

### 3.删除TYPE中的干扰项（包括一些形容词）

In [1]:
def TYPE_process(data_base,mylist):

    tt = data_base["TYPE"].apply(lambda x: x.replace(",", " "))
    for word in mylist:
        tt = tt.apply(lambda x: x.replace(word, ""))
    data_base["TYPE"] = tt.apply(lambda x: x.strip())
    
    return data_base

### 4.计算TYPE和TYPE之间的相似度
采用w2v和余弦相似度

In [ ]:
def w2v_similarity(data_base,test):
    
    df = pd.concat([test,data_base])
    document = df.loc[:, "TYPE"].drop_duplicates().to_list()
    
    name = [x.strip() for x in document]
    document = [x.strip().split(" ") for x in document]
    
    model = gensim.models.Word2Vec(sentences=document, min_count=1)
    word2v = []
    for i in document:
        try:
            vec_king = model.wv[i].mean(axis=0)
            word2v.append(list(vec_king))
        except:
            print(i)
    
    cos_sim = cosine_similarity(np.array(word2v))
    df_cos_sim = pd.DataFrame(cos_sim, index=name, columns=name)
    df_cos_sim = (df_cos_sim+1)/2
    
    return df_cos_sim

### 5.得到output

In [2]:
def get_output(each_test_candidates):
    
    #to_output
    mydf = pd.DataFrame()
    myindex = each_test_candidates.index
    k = 0
    for i in each_test_candidates:
        i["index"] = myindex[k]
        i["candidates"] = i.index
        mydf = pd.concat([mydf, i])
        k = k + 1
    arrays = [np.array(mydf["index"]), np.array(mydf["candidates"])]
    mm = pd.DataFrame(
        mydf.drop(columns=["index", "candidates"]).values,
        index=arrays,
        columns=mydf.drop(columns=["index", "candidates"]).columns,
    ).reset_index()
    
    return mm

### 6.如果test从data_base里面取

In [3]:
def test_from_database(data_base,size=0.02):
    
    train,test = train_test_split(data_base,test_size = size)
    
    return train,test.reset_index(drop = True)

In [4]:

import warnings
warnings.filterwarnings("ignore")

if __name__=="__main__":
    #读取VSIM表格
    data_base = pd.read_excel(r"C:\Users\zli242\Desktop\VOLVO\0815\data base.xlsx")
    axa = pd.read_excel(r"C:\Users\zli242\Desktop\VOLVO\packaging final_version\AXA P519.xlsx",header =4 ).iloc[4:,:]
    single_bom = pd.read_excel(r"C:\Users\zli242\Desktop\VOLVO\packaging final_version\Single bom.xlsx",header = 3).iloc[3:,:]
    plant = "CHN03"
    #清洗表格，删除样本数据的项目号
    data_base_processed = data_base_preprocess(data_base,test_newin = "")
    #清洗test
    test_processed = test_preprocess(axa, single_bom, plant)
    
    
    '''
    从data_base中提取test
    train,mytest = test_from_database(data_base_processed,size=0.02)
    '''
    
    # 删除样本数据和database的干扰项
    mylist = [
        "UNCOLORED",
        "KSOCOLOR",
        "CKDCOLOR",
        "CHARCOAL",
        "OFFBLACK",
        "CHARCOAL SOLID",
        "DAWN",
        "BRIGHT",
        "CHROME",
        "GRAINED",
        "PAINTED",
        "CARDAMOM",
        "BLACK",
        "WHITE",
        "WOOD",
        "LIGHT ASH",
        "LIGHT GUI",
        "METAL",
        "LUMIERE",
        "FOG MELANGE",
    ]
    train = TYPE_process(data_base_processed,mylist)
    mytest = TYPE_process(test_processed , mylist)
    """
    train = TYPE_process(train,mylist)
    mytest = TYPE_process(mytest , mylist)
    """
    
    df_cos_sim = w2v_similarity(train,mytest)
    
    
    #跑流程图, 
    each_test_candidates =mytest.apply(lambda x :find_best_candidates(x,train,new_in = []), axis=1)
    
    output = get_output(each_test_candidates)

NameError: name 'pd' is not defined

### 若从data_base中随意选取3000个，检测正确率

In [ ]:
#to_test
output["single_volume"] = output["LENGTH(MM)"] * output["WIDTH(MM)"] * output["HEIGHT(MM)"] / output["UL"]
mytest["single_volume"] = mytest["LENGTH(MM)"] * mytest["WIDTH(MM)"] * mytest["HEIGHT(MM)"]/mytest["UL"]
output["true_volume"] = [i for i in mytest["single_volume"] for j in range(3)]

output["diff"] = abs(output["true_volume"] - output["single_volume"]) / output["true_volume"]

summary = output.groupby("level_0").agg({"diff":"min"})

sum(summary["diff"] == 0)
sum((summary["diff"] > 0) & (summary["diff"] < 0.3))
sum((summary["diff"] > 0) & (summary["diff"] < 0.2))

SUMMARY = output.loc[output["level_1"] == 0]
sum(SUMMARY["diff"] == 0)
sum((SUMMARY["diff"] > 0) & (SUMMARY["diff"] < 0.3))
sum((SUMMARY["diff"] > 0) & (SUMMARY["diff"] < 0.2))

mm_big = output.loc[output["true_volume"] >= 1e6]

summary_big = mm_big.groupby("level_0").agg({"diff": "min"})
sum(summary_big["diff"] == 0)
sum((summary_big["diff"] > 0) & (summary["diff"] < 0.3))
sum((summary_big["diff"] > 0) & (summary["diff"] < 0.2))

mm_big = output.loc[output["true_volume"] >= 1e6]
SUMMARY_big = mm_big.loc[mm_big["level_1"] == 0]
sum(SUMMARY_big["diff"] == 0)
sum((SUMMARY_big["diff"] > 0) & (SUMMARY_big["diff"] < 0.3))
sum((SUMMARY_big["diff"] > 0) & (SUMMARY_big["diff"] < 0.2))